## 003. IMDB
- 영화 리뷰 긍/부정 판단 위한 sentiment analysis dataset
- 25000 train data, 25000 test data로 구성
- torxhtext를 이용해 데이터 다운로드하기

``` Python
! pip install torchtext=0.15.2 # 자연어 처리 작업
! pip install portalocker=2.7.0 # 여러 프로세스가 동일한  파일을 동시에 접근할 때 충돌 방지
! pip install accelerate -U # pytorch 모델의 학습 및 평가 가속화
```

In [1]:
# 위까지 실행 후 restart
from torchtext.datasets import IMDB

train_iter = IMDB(split='train')
test_iter = IMDB(split='test')



In [2]:
import random

random.seed(6)

# xxx_iter를 리스트로 변경
train_list = list(train_iter)
test_list = list(test_iter)

# 리스트 중 1000개씩 랜덤샘플링
train_list_small = random.sample(train_list, 1000)
test_list_small = random.sample(test_list, 1000)

# 각 변수의 첫번째 원소 출력
print(train_list_small[0])
print(test_list_small[0])

(2, "I LOVED this movie! I am biased seeing as I am a huge Disney fan, but I really enjoyed myself. The action takes off running in the beginning of the film and just keeps going! This is a bit of a departure for Disney, they don't spend quite as much time on character development (my husband pointed this out)and there are no musical numbers. It is strictly action adventure. I thoroughly enjoyed it and recommend it to anyone who loves Disney, be they young or old.")
(1, 'This was an abysmal show. In short it was about this kid called Doug who guilt-tripped a lot. Seriously he could feel guilty over killing a fly then feeling guilty over feeling guilty for killing the fly and so forth. The animation was grating and unpleasant and the jokes cheap. <br /><br />It aired here in Sweden as a part of the "Disney time" show and i remember liking it some what but then i turned 13.<br /><br />I never got why some of the characters were green and purple too. What was up with that? <br /><br />Tru

## 005. label encoding
위의 데이터셋은 (텍스트, 레이블) 형태로 구성되어 있으며, 레이블의 경우 2는 긍정, 1은 부정으로 구성되어 있다. 긍정을 1, 부정을 0으로 바꾸고 이걸 각각 train_texts, train_labels, test_texts, test_labels에 저장하라

In [3]:
train_texts  = []
train_labels = []
test_texts = []
test_labels = []

In [4]:
for label, text in train_list_small:
    train_labels.append(1 if label==2 else 0)
    train_texts.append(text)

In [5]:
for label, text in test_list_small:
    test_labels.append(1 if label==2 else 0)
    test_texts.append(text)

In [6]:
print(train_labels[0])
print(train_texts[0])

1
I LOVED this movie! I am biased seeing as I am a huge Disney fan, but I really enjoyed myself. The action takes off running in the beginning of the film and just keeps going! This is a bit of a departure for Disney, they don't spend quite as much time on character development (my husband pointed this out)and there are no musical numbers. It is strictly action adventure. I thoroughly enjoyed it and recommend it to anyone who loves Disney, be they young or old.


## 006. 학습 및 검증 데이터 분리
1000개를 8:2로 나누어라

In [7]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=3)

print(len(train_labels))
print(len(train_texts))

800
800


## 007. 토크나이징 및 인코딩
위에서 추출한 train, val, test 데이터를 pretrained distilbert-base-uncased 모델에 투입하기 위해 토크나이저를 사용해서 인코딩하라

In [8]:
from transformers import DistilBertTokenizerfast #DistilBertTokenizerfast

tokenizer = DistilBertTokenizerfast.from_pretrained('distilbert-base-uncased')

ImportError: cannot import name 'DistilBertTokenizerfast' from 'transformers' (c:\Users\User\anaconda3\envs\kaggle\Lib\site-packages\transformers\__init__.py)

In [ ]:
# 토크나이저 실행
train_encodings = tokenizer(train_texts, truncation=True, padding=True) # truncation: 모델의 default max_length를 넘는 입력 부분을 더 이상 받지 않고 절단함
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

print(train_encodings['input_ids'][0][:5])

TypeError: 'DistilBertTokenizer' object is not callable

In [ ]:
print(train_encodings.keys())

In [ ]:
print(train_encodings['input_ids'][0][:5])
print(tokenizer.decode(train_encodings['input_ids'][0][:5]))

## 008. 데이터세트 클래스 생성
- torch.utils.data.Dataset을 상속하는 IMDBDataset이라는 클래스 작성하기
- 문제 007의 imdb 데이터셋에서 학습한 train/val/test encodings를 입력해서 클래스 인스턴스화하기

cf. __init__은 인스턴스화에 생기는 generator이다. generator는 클래스로 객체를 생성할 때 자동으로 호출되는 메서드로, 객체를 미리 설정한 값으로 초기화한다. 그리고 클래스를 구성하는 arguments, variable, function을 정의한다.


In [ ]:
import torch

class IMDBDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # {'key':'value'} 형태의 딕셔너리 구조
        # val[idx], label[idx]의 데이터를 파이토치 텐서로 변환
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [ ]:
train_dataset = IMDBDataset(train_encodings, train_labels)
val_dataset = IMDBDataset(val_encodings, val_labels)
test_dataset = IMDBDataset(test_encodings, test_labels)

test_dataset

In [ ]:
# 각 데이터셋 확인: iterable 객체이므로 for문 사용가능
for i in train_dataset:
    print(i.keys())
    break


- input_ids: 단어 ID
- attention_mask: 패딩 위치(실제 입력 토큰과 패딩 토큰 구분)
- labels

## 사전학습 모델 불러오기

In [ ]:
from transformers import DistilBertForSequenceClassification

# 1. pretrained model 불러오기
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

model

100%|██████████| 267967963/267967963 [09:01<00:00, 494745.97B/s] 


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (dropout): Dropout(p=0.1, inplace=False)
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
   

In [ ]:
# 2. training_argument 설정
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./result',
    num_train_epochs = 8,
    per_device_train_batch_size = 16, # 학습시 디바이스별 미니 배치 수
    per_device_eval_batch_size = 64, # 평가시 ~
    warmup_steps = 500, # 학습률 스케줄링용 웜업 스텝수
    weight_decay=0.01, # 가중치 감소 강도
    logging_dir = './logs',
    logging_steps=10,
)

In [ ]:
# 3. 모델을 GPU에 전송
import torch

torch.cuda.is_available()


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)